In [1]:
import sys
sys.path.append('../scripts')
import data_loader

In [2]:
import os
import numpy as np
from scipy.misc import imsave
import tensorflow as tf
from tensorflow.contrib.layers.python import layers
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
import eval

In [6]:
"""Internal testing function."""
data_loader_obj = data_loader.DataLoader([100], test=True)
batch_iter = data_loader_obj.train_batch_iter(batch_size=100, num_epochs=1)

# Fake variables for testing
session = None
samples_op = None
y_samples_ph = 'y_ph'
z_samples_ph = 'z_ph'
num_randos = 1
class FakeSession:
    def run(self, dummy_op, feed_dict):
            yield 1.5 * feed_dict['y_ph'][0][1] + 3 + 10 * feed_dict['z_ph'][0]
session = FakeSession()    

In [8]:
p0_data, p1_data = eval.prepare_energy_data(batch_iter)

In [24]:
p0_data.output

[6736.7465298370034,
 3169.9961596324015,
 7978.6866132295636,
 3916.6971406359889,
 4589.2547533133902,
 1548.4648920214152,
 8377.138724970504,
 5883.0447637442967,
 8671.2414516658137,
 8595.089288617186,
 5312.1811084416304,
 2676.0355989429336,
 2033.4155420570069,
 2226.5471687436566,
 7456.1777514413379,
 1582.085485995598,
 8301.3904136687615,
 7781.1588651597503,
 7426.1588319401571,
 7463.0209137558459,
 6480.85058542966,
 7003.7981673558006,
 2367.9462827580787,
 3919.5306503843922,
 6032.2410582896218,
 1915.7854253052983,
 7366.1884116023866,
 6178.3610294656619,
 6857.9217181191443,
 3632.3515411081835,
 5469.1307931281017,
 2514.901291888064,
 7595.2841576285409,
 4930.5426565677762,
 5502.7518028332706,
 3671.450200813003,
 8544.8301306276499,
 5364.2700838027977,
 2626.3481413970371,
 1965.4777332327756,
 2801.256385111506,
 989.088107780163,
 4060.250812035069,
 2972.0324471510271,
 8543.4766547333966,
 4362.2642253581871,
 4096.489863566595]

# Config

In [5]:
WIDTH = 20  # Image width (X,Y axes)
N_SAMPLE_BINS= 10 #Right now we have 5 different momentum bins (100..500) x two particle types.  

POINTS_PER_FILE = 10000
RNG = np.random.RandomState(322)

class DefaultConfig(object):
    batch_size = 512 #TODO: why not bigger?
    samples_batch_size = 100 #number of samples of each particle type and momentum bin.
    z = 100
    max_epochs = 250
    lr = 0.0002
    beta1=0.5
    momentum = 0.5
    decay_steps = 100000
    decay_factor = 0.96
    train_dir = '/scratch/cdg356/udon/exp/'
    slice_idx = 12
    data_partition = [80, 10, 10]
    test = False
    generator_hits = 4
    save_freq = 1000
    
config = DefaultConfig

In [6]:
momentum = range(100,600,100)*2
n_momentums = len(set(momentum))
particle_type = [0]*5+[1]*5
n_particles = len(set(particle_type))

y = np.array(zip(particle_type,momentum)*config.samples_batch_size)
z = np.random.uniform(-1, 1, size=(N_SAMPLE_BINS*config.samples_batch_size, config.z))

In [74]:
def compare_to_minibatch(input, comparison_size = 16, num_kernels=5, kernel_dim=3):
    '''
    take output of intermediate layer of the discriminator,
    and compare individual samples within the minibatch
    '''

    #take subset for comparison batch
    slice_start = tf.shape(input)[0] - comparison_size
    x = tf.random_shuffle(input)
    x = tf.slice(x,[slice_start,0],[-1,-1])

    #multiply discriminator layer by 3D tensor to produce matrix
    x = layers.fully_connected (x, num_kernels * kernel_dim)
    activation = tf.reshape(x, (-1, num_kernels, kernel_dim))

    #compute L1-distance between rows of matrix
    diffs = tf.expand_dims(activation, 3) - \
        tf.expand_dims(tf.transpose(activation, [1, 2, 0]), 0)
    abs_diffs = tf.reduce_sum(tf.abs(diffs), 2)

    #apply negative exponential
    minibatch_features = tf.reduce_sum(tf.exp(-abs_diffs), 2)
    return tf.concat(1, [input, minibatch_features])

# Build a dataflow graph.
x = tf.ones([5,15])
y = tf.zeros([5,15])
z = tf.concat(0,[x,y])

out = compare_to_minibatch(z, comparison_size = 4, num_kernels=5, kernel_dim=3)

# Construct a `Session` to execute the graph.
sess = tf.Session()

# Execute the graph and store the value that `e` represents in `result`.
result = sess.run(out)

ValueError: last dimension shape must be known but is None